<a href="https://colab.research.google.com/github/leoipp/CENIBRA-GEO/blob/main/Rotina_de_indica%C3%A7%C3%A3o_de_anomalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?id=1gz4vgIvjL_lxInS--BPQ_w7hEDde2oRS" width=130 height=130 align="left">

#‎‎‎ Rotina de Indicação de Anomalias | Áreas Fomento.
#‎ Celulose Nipo-Brasileira S.A - CENIBRA.
---
---
> ‎ Rotina de detecção de anomalias, áreas com potencial distúrbio florestal. O script é baseado fundamentalmente no índice de vegetação por diferença normalizada (*IVDN* ou *NDVI* em inglês). O NDVI é uma medida radiométrica adimensional criada para distinguir informações espectrais da vegetação em relação a demais superfícies.

> Os valores do NDVI podem variar de *-1 a 1*, sendo *1* representado por maior densidade e vigor de cobertura vegetal da área amostrada. A maior amplitude do índice reflete um maior detalhamento das áreas vegetadas em contraste com o solo exposto e/ou impermeabilizados.

> As imagens de satélite foram obtidas pelo programa espacial *Copernicus* com a missão *Sentinel 2*, que consiste em 2 satélites orbitando a terra. O objetivo é monitorar a variabilidade das condições da superficie terrestre, com imagens em latitude média em espaçamentos de 2-3 dias e resolução espacial para as bandas *B4, B3, B2 e B8(NIR)* de *10 metros*.



## 1. Inicialização do ambiente
---




In [ ]:
#@markdown Instalação de pacotes não nativos do Colab.
!pip install geemap geopandas

In [1]:
#@markdown Importação de pacotes ao ambiente de execução.
import ee
import geemap
import fiona
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

from google.colab import drive

import os
import datetime as dt
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#@markdown Inicialização e autenticação para integração a plataforma Google Earth Engine.
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-7nz9t_J8TOlI3mIhsRXeP1Ja8FpRwF7eM9gpdyRXD8&tc=J_6p2OBMI2c11CQTQWSQfYqpiGz4p-ybJ4AgdjoDu1Q&cc=R7TJwToBi-Nh3nUOd1gdrs8Z5CBmlB-uDvG71UYuF_c

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWivKVn3MwvZnOW3AfsBZ6MR2R9zv9qOYMDZxUnejA8zSy0A7wBP_-s

Successfully saved authorization token.


## 2. Seleção de parâmetros.
---
> Os `.shp` da base de dados de talhões foram previamente armazenados na núvem para aumento de velocidade de importação do usuário.

O filtro criado s2cloudless para o dataset pode selecionar o limite de probabilidade de núvens, definindo uma máscara para núvens/não núvens.
A identificação do limite para a probabilidade de cobertura de núvens pode ser feita pela interpretação do usuário do **Histograma de Probabilidade de Núvens**.
Apenas máscaras agressivas irão permitir a remoção total de núvens da imagem, porém, podem eventualmente, remover pixels limpos.

Definição de parâmetros utilizados para filtrar a colgeção de imagens Sentinel 2 e determinar a identificação de núvens e projeção de sombra de núvens.

|Parâmetro | Tipo | Descrição |
| :-- | :-- | :-- |
| `AOI` | `ee.Geometry` | Área de interesse |
| `Data_inicio` | string | Início da coleção de imagens (inclusivo) |
| `data_final` | string | Fim da coleção de imagens (exclusivo) |
| `Filtro_nuvens` | integer | Percentual de cobertura máximo permitido na coleção de imagens |
| `Prob_limite_nuvens` | integer | Probabilidade de núvens (%); valores maiores que, sao considerados núvens |
| `Px_nao_agua` | float | Reflectância do vermelho proximo (NIR); valores menores que sao considerados pontos potenciais de sombras de núvens |
| `Projecao_nuvens` | float | Distância máxima (km) para procurar por sombras e bordas das núvens |
| `BUFFER` | integer | Distância (m) para dilatação das bordas de objetos identificados como núvens |

Quando parametrizando e avaliando máscaras de núvens para uma nova área, é uma boa prática identificar uma única data e limitar a extensão da regonal para minimizar requerimentos de processamento. 

**OBS:** Os dados de [Refecltância da superfície Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) e [Probabilidade de núvens Sentinel-2](https://developers.google.com/earth-engine/datasets/catalogCOPERNICUS_S2_CLOUD_PROBABILITY) são diferentes coleções de imagens. Cada coleção deve ser filtrada similarmente (ex. por data e extensão) e entao estas duas coleções filtradas devem ser sofrer junção.

Definir uma função para filtrar o SR e a coleção do S2 sem núves de acordo com a área de interesse e parâmetros de data, a junção ocorrerá pela propriedade de `system:index`. O resultando é uma cópia da coleção SR onde cada imagem tem uma nova propriedade `"s2cloudless"` no qual o valor é correspondente a imagem S2 sem núvens.

Definir uma função para adicionar pixels escuros, projeção de núvens e identificar sombras como bandas para o input de imagem S2 SR. **OBS:** O input da imagem necessita ser o resultado da função `ADD_BNuvem`, porque esta depende do conhecimento em consideração de cada pixel ser núvem ou não (Banda `clouds`)

In [92]:
# Camada de área de interesse
AOI_GN = ee.Geometry.Polygon(
        [[[-43.45230715090038,-19.123106898376598],
          [-41.99936525636913,-19.123106898376598],
          [-41.99936525636913,-18.016604678857952],
          [-43.45230715090038,-18.016604678857952]]], None)

# Camada de área de interesse
AOI_NE = ee.Geometry.Polygon(
        [[[-43.56741785025536,-20.25056297245866],
          [-42.54843591666161,-20.25056297245866],
          [-42.54843591666161,-18.98807690624204],
          [-43.56741785025536,-18.98807690624204]]], None)

# Camada de área de interesse
AOI_RD = ee.Geometry.Polygon(
        [[[-42.69306006150326,-20.421412226789364],
          [-41.68231787400326,-20.421412226789364],
          [-41.68231787400326,-18.3332108430093],
          [-42.69306006150326,-18.3332108430093]]], None)

# Acesso de dados no GEE
feature = "users/leoippef/BASE_FOMENTO"#@param {type:"string"}
regiao = "RD"#@param {type:"string"}
Areas_fomento = ee.FeatureCollection(feature).filter(ee.Filter.eq('REGIAO_ID', regiao))

In [93]:
Data_Inicial = '2022-04-01'#@param {type:"date"}
Data_Final = '2022-05-01'#@param {type:"date"}
Data_inicio = ee.Date.fromYMD(int(Data_Inicial.split('-')[0]),int(Data_Inicial.split('-')[1]),int(Data_Inicial.split('-')[2]))
Data_final = ee.Date.fromYMD(int(Data_Final.split('-')[0]),int(Data_Final.split('-')[1]),int(Data_Final.split('-')[2]))
Filtro_nuvens = 30 #@param {type:"slider", min:0, max:100, step:5}
Prob_limite_nuvens = 15 #@param {type:"slider", min:0, max:100, step:5}
Px_nao_agua = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
Projecao_nuvens = 1 #@param {type:"number"}
BUFFER = 10 #@param {type:"number"}

## 3. Definição de funções e metadados das imagens selecionadas.
---
> **OBS:** O output da função `Sentinel2`, será uma ImageCollection sem a aplicação dos filtros de máscara de núvens e projeção de sombras. Enquanto o `.median()` do valor `s2_sr_median` irá agregar todas imagens coletadas e realizar a média dos valores das bandas para o período especificado.




In [94]:
#@markdown Funções
def Sentinel2(aoi, Data_inicio, Data_final, Filtro_nuvens):
  
  # Importar e filtrar Sentinel 2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(Data_inicio, Data_final)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', Filtro_nuvens)))

    # Importar e filtrar sentinel 2 sem nuvens.
    s2_semnuvem_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(Data_inicio, Data_final))

    # Junção da coleção s2 sem nuvens com a coleção SR pela propriedade "system:index".
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{'primary': s2_sr_col,'secondary': s2_semnuvem_col, 'condition': ee.Filter.equals(**{'leftField': 'system:index','rightField': 'system:index'})}))


def Add_BNuvem(img):
  # Pegar imagens s2 sem nuvens, criando um subset de bandas de probabilidade.
  prob_nuvem = ee.Image(img.get('s2cloudless')).select('probability')

  # Condição de s2 sem nuvens pelo valor limite de probabilidade.
  enuvem = prob_nuvem.gt(Prob_limite_nuvens).rename('clouds')

  # Adicionar o layer de probabilidade e mascara de nuvens como bandas na imagem.
  return img.addBands(ee.Image([prob_nuvem, enuvem]))


def Add_BSombras(img):
    # Identificação de pixels de agua da banda SCL.
    naoagua = img.select('SCL').neq(6)

    # Identificação de pixels escuros NIR que nao sao agua (potencialmente pixels de sombra de nuvens).
    SR_BESCALA = 1e4
    pixels_escuros = img.select('B8').lt(Px_nao_agua*SR_BESCALA).multiply(naoagua).rename('pixels_escuros')

    # Determinar a direção de projeção de sombras de nuvens (Assumindo projeção UTM).
    Azimuth_sombreado = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Projetar sombras de nuvens para distancia especificada pelo input de Projecao_nuvens.
    projecao_nuvem = (img.select('clouds').directionalDistanceTransform(Azimuth_sombreado, Projecao_nuvens*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identificar a interseção de pixels escuros com projeção de sombra de nuvens.
    sombras = projecao_nuvem.multiply(pixels_escuros).rename('sombras')

    # Adicionar pixels escuros, projeção de nuvens, e sombras identificadas como bandas na imagem.
    return img.addBands(ee.Image([pixels_escuros, projecao_nuvem, sombras]))


def Add_BNS(img):
    # Adicionar componentes da banda de nuvens.
    img_nuvem = Add_BNuvem(img)

    # Adicionar componente de sombra de nuvens.
    img_nuvem_shadow = Add_BSombras(img_nuvem)

    # Combinar mascara de nuvens e sombra, selecionar nuvem e sombra como valores 1 e o resto como 0.
    e_sombreamento = img_nuvem_shadow.select('clouds').add(img_nuvem_shadow.select('sombras')).gt(0)

    # Remover pequenas areas de sombras de nuvens e dilatar os pixels remanescentes pelo input do Buffer.
    # 20 metros de escala para maior velocidade, assumindo que a analise nao necessite de 10 m de precisão.
    e_sombreamento = (e_sombreamento.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 10})
        .rename('cloudmask'))

    # Adicionar uma máscara final de sombreamento de nuvens na imagem.
    return img_nuvem_shadow.addBands(e_sombreamento)


def App_CSmask(img):
    # Subset a banda da mascara de nuvens e inverter, assim nuvem/sombras são 0, e o restante 1.
    nao_sombras = img.select('cloudmask').Not()

    # Subset das bandas de reflectancia e atualização das mascaras, retornando o resultado.
    return img.select('B.*').updateMask(nao_sombras)

def addNDVI (image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

if regiao == "NE":
  # Obter coleção de imagem crua do Sentinel 2.
  s2 = Sentinel2(AOI_NE, Data_inicio, Data_final, Filtro_nuvens)
  # s2 = s2.map(addNDVI)

  # Aplicação de mascara de nuvens/sombras na Coleção de Imagens.
  # Converter uma coleção de imagens em uma unica imagem atraves da funcao median().
  s2_sr_cld_col_eval_disp = s2.map(Add_BNS)
  s2_sr_median = (s2.map(Add_BNS).map(App_CSmask).map(addNDVI).median()) 

if regiao == "GN":
  # Obter coleção de imagem crua do Sentinel 2.
  s2 = Sentinel2(AOI_GN, Data_inicio, Data_final, Filtro_nuvens)
  # s2 = s2.map(addNDVI)

  # Aplicação de mascara de nuvens/sombras na Coleção de Imagens.
  # Converter uma coleção de imagens em uma unica imagem atraves da funcao median().
  s2_sr_cld_col_eval_disp = s2.map(Add_BNS)
  s2_sr_median = (s2.map(Add_BNS).map(App_CSmask).map(addNDVI).median())
if regiao == "RD":
  # Obter coleção de imagem crua do Sentinel 2.
  s2 = Sentinel2(AOI_RD, Data_inicio, Data_final, Filtro_nuvens)
  # s2 = s2.map(addNDVI)

  # Aplicação de mascara de nuvens/sombras na Coleção de Imagens.
  # Converter uma coleção de imagens em uma unica imagem atraves da funcao median().
  s2_sr_cld_col_eval_disp = s2.map(Add_BNS)
  s2_sr_median = (s2.map(Add_BNS).map(App_CSmask).map(addNDVI).median())



In [95]:
#@markdown Metadata da coleção de imagens adquirirdas no Google Earth Engine API
# Metadata da coleção de imagens adquirirdas no Google Earth Engine API.
# Obter o range de datas com formato já convertido.
range = s2.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
Data_inicio = ee.Date(range.get('min')).format("YYYY-MM-dd")
Data_final = ee.Date(range.get('max')).format("YYYY-MM-dd")
print(f'Período de obtenção: Início em {Data_inicio.getInfo()} / Fim em {Data_final.getInfo()}')
# Obter o numero de imagens na coleção.
print(f'Para os parametros selecionados, você obteve: {s2.size().getInfo()} Imagens.') 


Período de obtenção: Início em 2022-04-18 / Fim em 2022-04-28
Para os parametros selecionados, você obteve: 18 Imagens.


## 4. Amostragem de NDVI para talhões.
---
> Para extrair os valores `NDVI` o Google drive precisa estar adicionado a pasta do notebook. Para isto, disponibilize o token de acesso ao rodar `drive.mount()`.

> **OBS:** A exportação dos shapefiles `NDVI` aconteceram em pasta previamente criada pelo usuário, esta deve ser o caminho `work_dir`. Caso nao tenha uma pasta identificada para exportação, a mesma não ocorrerá.



> **OBS.2:**: Atualmente a função `zonal_statistics()` pode calcular um tipo por vez, sendo estes `MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM`.

> **A exportação dos `.shp` podem levar aproximadamente 15 min ou mais**

> Certamente quando talhões apresentarem valores de `NaN` para o atributo de **NDVI** a área amostrada está **contaminada por densa cobertura de núvens**. A aplicação da máscara de núvens irá suprimir a área deixando um "buraco" na imagem, portanto, não obtendo valor.

> Neste caso, recomenda-se a utilização de outra coleção de imagens de salite, como exemplo o **MODIS**, porém a resolução da banda será prejudicada, neste exemplo será de 250 metros de resolução espacial.

> A princípio haverá somente análise em talhões maduros (maiores que dois anos desde o plantio).

> Pode-se avaliar talhões jovens que já sofreram análise de sobrevivencia, caso haja necessidade.



In [ ]:
NDVI = s2_sr_median.normalizedDifference(['B8', 'B4']).rename('NDVI')
# Seleção de pasta destino (pré criada) para exportação dos .shp.
work_dir = "/content/drive/MyDrive/Fomento_CENIBRA"#@param {type: 'string'}

# input do nome do output da camada de valores extraidas a partir da imagem NDVI.
# Este passo pode levar alguns minutos para completar.
out_shp_mean = os.path.join(work_dir, 'ZONAL_MEAN.shp')
geemap.zonal_statistics(NDVI, Areas_fomento, out_shp_mean, statistics_type='MEAN', scale=10)


out_shp_min_max = os.path.join(work_dir, 'ZONAL_MIN_MAX.shp')
geemap.zonal_statistics(NDVI, Areas_fomento, out_shp_min_max, statistics_type='MIN_MAX', scale=10)

out_shp_variance = os.path.join(work_dir, 'ZONAL_VAR.shp')
geemap.zonal_statistics(NDVI, Areas_fomento, out_shp_variance, statistics_type='VARIANCE', scale=10)

In [ ]:
#@markdown Talhoes
# Abertura dos .shp anteriormente exportados para o drive.
ndvi_mean = gpd.read_file('/content/drive/MyDrive/Fomento_CENIBRA/ZONAL_MEAN.shp')
ndvi_min_max = gpd.read_file('/content/drive/MyDrive/Fomento_CENIBRA/ZONAL_MIN_MAX.shp')
ndvi_variance = gpd.read_file('/content/drive/MyDrive/Fomento_CENIBRA/ZONAL_VAR.shp')

ndvi_min_max['REF_ID'] = ndvi_min_max.REGIAO + ndvi_min_max.ID_PROJETO + ndvi_min_max.LOTE + '-' + ndvi_min_max.TALHAO
# Criando uma copia para integração de colunas em um GeoDataFrame unico.
NDVI = ndvi_min_max.copy()


if 'mean' in ndvi_mean:
  # Conversão de formato str para datetime.
  NDVI['Date'] = pd.to_datetime(NDVI['DATA_DE_PL'])
  NDVI = NDVI.drop(columns=['DATA_DE_PL'])
  # Cria o campo idade em meses
  NDVI["IDADE_MESES"] = NDVI["Date"].apply(lambda x : round((dt.datetime.now() - x)/np.timedelta64(1,'M')))

  # Adição de colunas em diferetes .shp em uma unica.

  NDVI['mean'] = ndvi_mean['mean']
  NDVI['variance'] = ndvi_variance['variance']

  # Análise descritiva dos dados (n. de linhas e colunas e quantidade de NaN valores).
  print(f'O GeoDataFrame possui {(NDVI.shape)[0]} Linhas / {(NDVI.shape)[1]} Colunas')
  # print(NDVI.isnull().sum())

  # Checar o index de NDVI para NaN valores, caso exista.
  x = NDVI.loc[pd.isnull(NDVI["mean"]), :].index

  # Remover NaN valores (interseção com núvens NDVI=0)
  NDVI = NDVI[NDVI['mean'].notna()]
  print(' ******************************************************')
  print(' *************** REMOÇÃO DE NAN VALORES ***************')
  print(' ******************************************************')
  print(f'O GeoDataFrame possui {(NDVI.shape)[0]} Linhas / {(NDVI.shape)[1]} Colunas')

  # Seleção de talhoes jovens e maduros.
  Talhoes_maduros = NDVI[NDVI['IDADE_MESES'] > 24]
  Talhoes_jovens = NDVI[NDVI['IDADE_MESES'] <= 24]

  # Seleção de áreas saudáveis (NDVI MÉDIO >= .55).
  Nao_anomalos = Talhoes_maduros[Talhoes_maduros['mean'] >= 0.55]
  # Seleção de talhoes anomalos.
  Anomalos= Talhoes_maduros[Talhoes_maduros['mean']< 0.55].reset_index(drop=True)
  print(' ******************************************************')
  print(' ******************************************************')
  # Descrição de valores encontrando em áreas com potencial distúrbio.
  print(f'> Número de Talhões potencialmente com disturbio: {Anomalos["mean"].describe()[0]} \n> Valor de NDVI médio mais baixo entre Talhões: {Anomalos["mean"].describe()[3]} \n> Valor de NDVI médio mais alto entre Talhões: {Anomalos["mean"].describe()[7]} \n> Média de NDVI médio entre Talhões com potencial distúrbio: {Anomalos["mean"].describe()[1]}')
  print(' ******************************************************')
  print(' ******************************************************')
  Anomalos_T = Anomalos.copy()
  # REF ID de talhões com potencial distúrbio
  a = 0
  for i in Anomalos.REF_ID:
    a = a + 1
    print(f'ID de Referência do Talhão: {a}/{len(Anomalos)} - Com Potencial Distúrbio: {i}')



  Anomalos = Anomalos.drop(columns=['Date', 'IDADE_MESES'])

else:  
  print(' *******************************************************************************')
  print('ALTA DENSIDADE DE NÚVENS RECOBRINDO A AOI PARA O PERÍODO E FILTROS ESPECIFICADOS')
  print(' *******************************************************************************')
  print('************************ SEGUINDO PARA AMOSTRAGEM POR PONTOS *******************')

## 5. Amostragem de NDVI para pontos.
---
> As amostras de 50 x 50m foram criadas previamente no *software ArcGIS PRO*, pela ferramente `FishNet`. As amostras foram então `'clipadas'` pelo poligono dos talhões com buffer de -12 metros, para evitar amostragem de pontos em pixel de estrada.


> As propriedades dos talhões e pontos sofreram `'junção espacial'` para então exportação para núvem, armazenada no Google Earth Engine para maior performance de captura dos dados.

> Os passos seguem a extração de valores como no passo **4.** A FeatureCollection `Amostragem_px`, foi previamente armazenada em nuvem para aumento de performance do script na extração de valores de `NDVI`.

> Haverá também a extração de valores de presença de núvens em cada pixel e a probabilidade do pixel conter a cobertura de núvem cirrus.

> **A exportação dos `.shp` podem levar aproximadamente 5 min ou mais**

> Para facilitar a identificação de áreas com potencial disturbio, iremos localizar cada talhão e seu respectivo poligono pelo `groupby(Anomalos.REF_ID)`, que irá reporesentar a média de valores de NDVI para os pontos daquele local.



In [ ]:
# Calculo de NDVI
NDVI = s2_sr_median.normalizedDifference(['B8', 'B4']).rename('NDVI')
# Importação dos pontos de amostragem e filtro de feições TA03 e regioes RDBOBA
feature = "users/leoippef/BASE_AMOSTRAS_FOMENTO"#@param {type: "string"}
Amostragem_px = ee.FeatureCollection(feature).filter(ee.Filter.eq('REGIAO_ID', regiao))
# Selecao de valores e probabilidade de núvem para filtrar dados
img = s2_sr_cld_col_eval_disp.mosaic()
clouds = img.select(['clouds', 'probability'])
out_shp = os.path.join(work_dir, 'Amostragem_NUVEM.shp')
geemap.extract_values_to_points(Amostragem_px, clouds, out_shp, scale=10)

out_shp = os.path.join(work_dir, 'Amostragem_NDVIBUFFER.shp')
geemap.extract_values_to_points(Amostragem_px, NDVI, out_shp, scale=10)

In [ ]:
#@markdown Amostras
Amostras_NDVI = gpd.read_file('/content/drive/MyDrive/Fomento_CENIBRA/Amostragem_NDVIBUFFER.shp')
Amostras_nuvem = gpd.read_file('/content/drive/MyDrive/Fomento_CENIBRA/Amostragem_NUVEM.shp')

if "first" in Amostras_NDVI:

  Amostras_NDVI = Amostras_NDVI.rename(columns={'first': 'NDVI'})
  Amostras_NDVI['REF_ID'] = Amostras_NDVI.REGIAO + Amostras_NDVI.ID_PROJETO + Amostras_NDVI.LOTE + '-' + Amostras_NDVI.TALHAO


  # Junção de dados Nuvem com GeoDataFrame do NDVI
  Amostras_NDVI['nuvem'] = Amostras_nuvem['clouds']
  Amostras_NDVI['Prob_nuvem'] = Amostras_nuvem['probabilit']

  # Conversão de formato str para datetime.
  Amostras_NDVI['Date'] = pd.to_datetime(Amostras_NDVI['DATA_DE_PL'])
  # Amostras_NDVI = Amostras_NDVI.drop(columns=['DATA_DE_PL'])
  # Cria o campo idade em meses
  Amostras_NDVI["IDADE_MESES"] = Amostras_NDVI["Date"].apply(lambda x : round((dt.datetime.now() - x)/np.timedelta64(1,'M')))


  # Análise descritiva dos dados (n. de linhas e colunas e quantidade de NaN valores).
  print(f'O GeoDataFrame possui {(Amostras_NDVI.shape)[0]} Pontos de Amostra / {(Amostras_NDVI.shape)[1]} Atributos')
  #print(Amostras_NDVI.isna().sum())
  print(' ******************************************************')
  print(' ******************************************************')

  # Remover NaN valores (interseção com núvens NDVI=0)
  Amostras_NDVI = Amostras_NDVI[Amostras_NDVI['NDVI'].notna()]
  print(' **************** REMOÇÃO DE NAN VALORES **************')
  print(' ******************************************************')
  print(f'O GeoDataFrame possui {(Amostras_NDVI.shape)[0]} Pontos de Amostra / {(Amostras_NDVI.shape)[1]} Atributos')
  print(' ******************************************************')
  print(' ******************************************************')



  # Separação de Talhoes maduros e Talhoes Jovens.

  Talhoes_maduros = Amostras_NDVI[Amostras_NDVI['IDADE_MESES'] > 24]
  Talhoes_jovens = Amostras_NDVI[Amostras_NDVI['IDADE_MESES'] <= 24]
  # Seleção de áreas saudáveis (NDVI MÉDIO >= .55).
  Nao_anomalos = Talhoes_maduros[Talhoes_maduros['NDVI'] >= 0.55]

  # Seleção de talhoes anomalos.
  Anomalos= Talhoes_maduros[(Talhoes_maduros['NDVI']< 0.55) & (Talhoes_maduros['nuvem'] == 0) & (Talhoes_maduros['Prob_nuvem'] < 35)].reset_index(drop=True)


  # Descrição de valores encontrando em áreas com potencial distúrbio.
  print(f'> Número de Pontos potencialmente com disturbio: {Anomalos["NDVI"].describe()[0]} \n> Valor de NDVI mais baixo entre Talhões: {Anomalos["NDVI"].describe()[3]} \n> Valor de NDVI mais alto entre Talhões: {Anomalos["NDVI"].describe()[7]} \n> Média de NDVI entre Talhões com potencial distúrbio: {Anomalos["NDVI"].describe()[1]}')

  print(' ******************************************************')
  print(' ******************************************************')

  Anomalias_group = Anomalos.groupby(Anomalos.REF_ID).mean().reset_index()
  print(' *********** AGRUPAMENTO DE AMOSTRAS POR REF_ID *******')
  print(' ******************************************************')
  print(f'O GeoDataFrame possui {(Anomalias_group.shape)[0]} Pontos de Amostra / {(Anomalias_group.shape)[1]} Atributos')
  print(' ******************************************************')
  print(' ******************************************************')
  print(' ')

  # Lista de referencia para extrair poligonos por ID
  REFID = list(Anomalias_group.REF_ID)
  # Criando uma copia para integração de colunas em um GeoDataFrame unico.
  NDVI = ndvi_min_max.copy()
  Anomalos_talhoes = gpd.GeoDataFrame()
  for i in REFID:
    for ii, n in NDVI.iterrows():
      if i == n.REF_ID:
        Anomalos_talhoes = Anomalos_talhoes.append(NDVI.loc[ii])
  # Input de CRS para GeoDataFrame (Caso queira representar em `geemap()`)
  Anomalos_talhoes.crs = "EPSG:4326"


  # Identificar talhoes com média de NDVI que foram computados, mas nao estao presentes nesta amostragem
  for i in list(Anomalos_talhoes.REF_ID):
    for ii, n in Anomalos_T.iterrows():
      if i == n.REF_ID:
        Anomalos_T = Anomalos_T.drop(ii)

  Anomalos_T = Anomalos_T.drop(columns=['mean', 'variance', 'Date', 'IDADE_MESES'])
  Anomalos = Anomalos.drop(columns=['Date', 'IDADE_MESES'])

  # Concatenar areas de poligono 
  Areas_anomalia = pd.concat([Anomalos_talhoes, Anomalos_T])
  print('Processando cálculo de área para anomalias.....')

  # Calculo de área para as anomalias

  for i, n in Areas_anomalia.iterrows():
    df = gpd.GeoDataFrame()
    df = df.append(Areas_anomalia.loc[i])
    df.crs = "EPSG: 4326"
    aor = geemap.geopandas_to_ee(df)
    s2_ndvi = s2_sr_median.select('NDVI')
    sample = s2_ndvi.sample(aor, 10)
    low = sample.filter(ee.Filter.lt('NDVI', 0.55))
    low = low.aggregate_stats('NDVI').getInfo()
    area = low['total_count']*100
    Areas_anomalia.loc[i, 'Area_anomalia'] = area


  # Exportar pontos anomalos e poligonos para formato KML
  fiona.supported_drivers['ESRI Shapefile'] = 'rw'
  Areas_anomalia.to_file(f'Anomalos_talhoes_{regiao}.shp', driver='ESRI Shapefile')
  # Anomalos.to_file('Anomalos_pontos.shp', driver='ESRI Shapefile')

  print(f'Quantidade final de talhões: {len(Areas_anomalia)}')
  print(' ******************************************************')
  print(' ******************************************************')

  if regiao == "NE":
    out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_NE), description=out_nir, folder='export', scale=10)
    out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_NE), description=out_ndvi, folder='export', scale=10)
    out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_NE), description=out_rgb, folder='export', scale=10)
  if regiao == "GN":
    out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_GN), description=out_nir, folder='export', scale=10)
    out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_GN), description=out_ndvi, folder='export', scale=10)
    out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_GN), description=out_rgb, folder='export', scale=10)
  if regiao == "RD":
    out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_RD), description=out_nir, folder='export', scale=10)
    out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_RD), description=out_ndvi, folder='export', scale=10)
    out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
    geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_RD), description=out_rgb, folder='export', scale=10)


else:  
  print(' *******************************************************************************')
  print('ALTA DENSIDADE DE NÚVENS RECOBRINDO A AOI PARA O PERÍODO E FILTROS ESPECIFICADOS')
  print(' *******************************************************************************')
  print('************************ NENHUMA AMOSTRA SELECIONADA *******************')


  if "first" not in Amostras_NDVI and 'mean' in ndvi_mean :
    Anomalos.crs = "EPSG:4326"
    fiona.supported_drivers['ESRI Shapefile'] = 'rw'
    Anomalos.to_file(f'Anomalos_talhoes_{regiao}.shp', driver='ESRI Shapefile')
    # Anomalos.to_file('Anomalos_pontos.shp', driver='ESRI Shapefile')

    # Calculo de área para as anomalias

    for i, n in Anomalos.iterrows():
      df = gpd.GeoDataFrame()
      df = df.append(Anomalos.loc[i])
      df.crs = "EPSG: 4326"
      aor = geemap.geopandas_to_ee(df)
      s2_ndvi = s2_sr_median.select('NDVI')
      sample = s2_ndvi.sample(aor, 10)
      low = sample.filter(ee.Filter.lt('NDVI', 0.55))
      low = low.aggregate_stats('NDVI').getInfo()
      area = low['total_count']*100
      Anomalos.loc[i, 'Area_anomalia'] = area

    print(f'Quantidade final de talhões: {len(Anomalos)}')
    print(' ******************************************************')
    print(' ******************************************************')

    if regiao == "NE":
      out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_NE), description=out_nir, folder='export', scale=10)
      out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_NE), description=out_ndvi, folder='export', scale=10)
      out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_NE), description=out_rgb, folder='export', scale=10)
    if regiao == "GN":
      out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_GN), description=out_nir, folder='export', scale=10)
      out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_GN), description=out_ndvi, folder='export', scale=10)
      out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_GN), description=out_rgb, folder='export', scale=10)
    if regiao == "RD":
      out_nir = (f'RGB_NIR_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B8','B4', 'B3', 'B2']).clip(AOI_RD), description=out_nir, folder='export', scale=10)
      out_ndvi = (f'NDVI_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['NDVI']).clip(AOI_RD), description=out_ndvi, folder='export', scale=10)
      out_rgb = (f'RGB_{s2.size().getInfo()}_{regiao}_{Data_Inicial}')
      geemap.ee_export_image_to_drive(s2_sr_median.select(['B4', 'B3', 'B2']).clip(AOI_RD), description=out_rgb, folder='export', scale=10)